# 🐺 WOLF PACK RESEARCH - SETUP
## Foundation for All Research Notebooks

**Purpose:** Load libraries, define universe, create helper functions

**Usage:** Run this notebook first, OR copy these cells to the top of each research notebook

---

In [ ]:
# === IMPORTS ===
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Pandas display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# Matplotlib style
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Libraries loaded")

In [ ]:
# === THE REAL UNIVERSE - NOT GENERIC BULLSHIT ===
# These are the ACTUAL plays. The ones that moved. The ones moving. The ones about to move.

UNIVERSE = {
    # MEMORY - MU +20% on analyst upgrades, DRAM squeeze, HBM sold out through 2026
    'MEMORY': ['MU', 'INTC', 'AMD', 'NVDA'],
    
    # AI INFRA - APLD +18% on earnings + $5B deal + short squeeze
    'AI_INFRA': ['APLD', 'CORZ', 'WULF', 'HUT', 'CLSK', 'CIFR', 'IREN', 'MARA', 'RIOT'],
    
    # NUCLEAR - OKLO +17% on Meta deal, VST +11% sympathy. The REAL AI power play.
    'NUCLEAR': ['SMR', 'NNE', 'OKLO', 'VST', 'CEG', 'CCJ', 'UEC', 'UUUU', 'LEU'],
    
    # DEFENSE - Trump 50% budget increase, KTOS (Tyr owns), RCAT drones
    'DEFENSE': ['KTOS', 'RCAT', 'PLTR', 'BBAI', 'AVAV', 'LMT', 'RTX'],
    
    # SPACE - RKLB $805M contract, space exec order Dec 2025
    'SPACE': ['RKLB', 'LUNR', 'RDW', 'ASTS', 'SPIR', 'BKSY'],
    
    # LIDAR / AV - AEVA +51% week, led whole sector on NVIDIA partnership
    'LIDAR_AV': ['AEVA', 'OUST', 'INVZ', 'LAZR', 'MVIS'],
    
    # AI SOFTWARE - SOUN CES catalyst, voice AI is the interface
    'AI_SOFTWARE': ['SOUN', 'AI', 'PATH', 'UPST'],
    
    # QUANTUM - Extreme volatility, RGTI +1000% then crashed. Pattern testing.
    'QUANTUM': ['IONQ', 'RGTI', 'QBTS', 'QUBT'],
    
    # FINTECH - HOOD 13 insider transactions! SOFI profitable with bank charter
    'FINTECH': ['SOFI', 'HOOD', 'AFRM', 'NU'],
    
    # BIOTECH - NTLA (Tyr owns) JP Morgan conference this week
    'BIOTECH': ['NTLA', 'CRSP', 'BEAM', 'EDIT', 'RXRX', 'DNA'],
    
    # CANNABIS - TLRY (Tyr owns) just beat earnings +7% AH
    'CANNABIS': ['TLRY', 'CGC', 'SNDL'],
    
    # EMERGING - Second order plays, next runners before they run
    'EMERGING': ['SKYT', 'KULR', 'GEVO', 'PLUG', 'BE', 'FCEL'],
}

# Build full ticker list
ALL_TICKERS = list(set([t for sector in UNIVERSE.values() for t in sector]))

# Reverse lookup - which sector is each ticker in
TICKER_TO_SECTOR = {}
for sector, tickers in UNIVERSE.items():
    for t in tickers:
        TICKER_TO_SECTOR[t] = sector

# The big winners we need to reverse engineer
BIG_WINNERS = {
    'MU': {'move': '+20%', 'when': 'Week 1 Jan 2026', 'why': 'Analyst upgrades, DRAM squeeze'},
    'APLD': {'move': '+18%', 'when': 'Jan 7-10', 'why': '$5B hyperscaler deal + earnings + short squeeze'},
    'OKLO': {'move': '+17%', 'when': 'Jan 9', 'why': 'Meta nuclear deal announced'},
    'VST': {'move': '+11%', 'when': 'Jan 9', 'why': 'Sympathy to OKLO/Meta deal'},
    'AEVA': {'move': '+51%', 'when': 'Week of Jan 6', 'why': 'NVIDIA partnership, CES'},
    'TLRY': {'move': '+7% AH', 'when': 'Jan 8', 'why': 'Earnings beat $218M revenue'},
}

# Current holdings (Tyr's positions)
CURRENT_HOLDINGS = ['APLD', 'KTOS', 'TLRY', 'NTLA']

print(f"🐺 REAL UNIVERSE LOADED: {len(ALL_TICKERS)} tickers across {len(UNIVERSE)} sectors")
print()
print("SECTORS:")
for sector, tickers in UNIVERSE.items():
    print(f"  {sector}: {', '.join(tickers)}")
print()
print("📈 BIG WINNERS TO REVERSE ENGINEER:")
for ticker, data in BIG_WINNERS.items():
    print(f"  {ticker}: {data['move']} - {data['why']}")
print()
print(f"💼 CURRENT HOLDINGS: {', '.join(CURRENT_HOLDINGS)}")

In [ ]:
# === HELPER FUNCTIONS - THE GOOD STUFF ===

def get_data(ticker, period='3mo'):
    """Pull historical data - our raw material"""
    try:
        df = yf.Ticker(ticker).history(period=period)
        if len(df) > 0:
            df['ticker'] = ticker
            df['sector'] = TICKER_TO_SECTOR.get(ticker, 'UNKNOWN')
            return df
    except Exception as e:
        print(f"⚠️  Error loading {ticker}: {e}")
    return None

def get_all_data(tickers=None, period='3mo', verbose=True):
    """Get data for entire universe or subset"""
    if tickers is None:
        tickers = ALL_TICKERS
    
    frames = []
    if verbose:
        print(f"Loading {len(tickers)} tickers...")
    
    for ticker in tickers:
        df = get_data(ticker, period)
        if df is not None and len(df) > 20:
            frames.append(df)
        if verbose:
            print(".", end="", flush=True)
    
    if verbose:
        print(f" Done! {len(frames)}/{len(tickers)} loaded")
    
    if len(frames) == 0:
        return pd.DataFrame()
    
    return pd.concat(frames)

def rsi(prices, period=14):
    """Calculate RSI indicator"""
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def add_features(df):
    """Add ALL the indicators we might care about"""
    df = df.copy()
    
    # Basic returns
    df['daily_ret'] = df['Close'].pct_change()
    df['intraday_ret'] = (df['Close'] - df['Open']) / df['Open']
    
    # Volume analysis
    df['vol_avg_20'] = df['Volume'].rolling(20).mean()
    df['vol_ratio'] = df['Volume'] / df['vol_avg_20']
    df['vol_trend'] = df['Volume'].rolling(5).mean() / df['Volume'].rolling(20).mean()
    
    # RSI
    df['rsi'] = rsi(df['Close'])
    
    # Momentum - past performance
    df['ret_1d_past'] = df['Close'] / df['Close'].shift(1) - 1
    df['ret_3d_past'] = df['Close'] / df['Close'].shift(3) - 1
    df['ret_5d_past'] = df['Close'] / df['Close'].shift(5) - 1
    df['ret_10d_past'] = df['Close'] / df['Close'].shift(10) - 1
    df['ret_20d_past'] = df['Close'] / df['Close'].shift(20) - 1
    
    # Distance from highs
    df['high_5d'] = df['High'].rolling(5).max()
    df['high_20d'] = df['High'].rolling(20).max()
    df['high_60d'] = df['High'].rolling(60).max()
    df['dist_from_5d_high'] = df['Close'] / df['high_5d'] - 1
    df['dist_from_20d_high'] = df['Close'] / df['high_20d'] - 1
    df['dist_from_60d_high'] = df['Close'] / df['high_60d'] - 1
    
    # Distance from lows
    df['low_20d'] = df['Low'].rolling(20).min()
    df['dist_from_20d_low'] = df['Close'] / df['low_20d'] - 1
    
    # Volatility
    df['volatility_10d'] = df['daily_ret'].rolling(10).std()
    df['volatility_20d'] = df['daily_ret'].rolling(20).std()
    
    # Gap analysis
    df['gap'] = df['Open'] / df['Close'].shift(1) - 1
    
    # Range
    df['daily_range'] = (df['High'] - df['Low']) / df['Close']
    df['range_avg'] = df['daily_range'].rolling(10).mean()
    
    # Moving averages
    df['sma_10'] = df['Close'].rolling(10).mean()
    df['sma_20'] = df['Close'].rolling(20).mean()
    df['sma_50'] = df['Close'].rolling(50).mean()
    df['above_sma20'] = df['Close'] > df['sma_20']
    df['dist_from_sma20'] = df['Close'] / df['sma_20'] - 1
    
    # Day of week
    df['dow'] = pd.to_datetime(df.index).dayofweek
    df['is_monday'] = df['dow'] == 0
    df['is_friday'] = df['dow'] == 4
    
    # Forward returns (what we're trying to predict)
    df['fwd_1d'] = df['Close'].shift(-1) / df['Close'] - 1
    df['fwd_3d'] = df['Close'].shift(-3) / df['Close'] - 1
    df['fwd_5d'] = df['Close'].shift(-5) / df['Close'] - 1
    df['fwd_10d'] = df['Close'].shift(-10) / df['Close'] - 1
    
    # Is this a big move day?
    df['is_big_up'] = df['daily_ret'] > 0.10
    df['is_big_down'] = df['daily_ret'] < -0.10
    
    return df

def print_stats(series, label=""):
    """Print summary statistics for a series"""
    clean = series.dropna()
    if len(clean) == 0:
        print(f"{label}: No data")
        return
    
    print(f"{label}:")
    print(f"  Count: {len(clean)}")
    print(f"  Mean: {clean.mean()*100:+.2f}%")
    print(f"  Median: {clean.median()*100:+.2f}%")
    print(f"  Win Rate: {(clean > 0).mean()*100:.1f}%")
    print(f"  Best: {clean.max()*100:+.2f}%")
    print(f"  Worst: {clean.min()*100:+.2f}%")

def load_master():
    """Load entire universe with all features"""
    data = get_all_data()
    frames = []
    for ticker in data['ticker'].unique():
        df = data[data['ticker'] == ticker].copy()
        df = add_features(df)
        frames.append(df)
    return pd.concat(frames)

print("✅ Helper functions loaded")
print()
print("QUICK FUNCTIONS:")
print("  get_data('MU', '3mo')        - Get single ticker")
print("  get_all_data()               - Get entire universe")
print("  add_features(df)             - Add all indicators")
print("  load_master()                - Load everything with features")
print("  print_stats(series, 'label') - Print return stats")

In [ ]:
# === LOAD ALL DATA ===
print("🐺 Loading entire universe...")
print("This takes 1-2 minutes. Be patient.\n")

all_frames = []
failed = []

for ticker in ALL_TICKERS:
    try:
        df = yf.Ticker(ticker).history(period='3mo')
        if len(df) > 20:
            df['ticker'] = ticker
            df['sector'] = TICKER_TO_SECTOR.get(ticker, 'UNKNOWN')
            df = add_features(df)
            all_frames.append(df)
            print("✓", end="", flush=True)
        else:
            failed.append(ticker)
            print(".", end="", flush=True)
    except:
        failed.append(ticker)
        print("X", end="", flush=True)

master = pd.concat(all_frames)
print(f"\n\n✅ LOADED {len(all_frames)} tickers, {len(master)} total rows")
print(f"Date range: {master.index.min().date()} to {master.index.max().date()}")
if failed:
    print(f"⚠️  Failed to load: {', '.join(failed)}")

---

## 🐺 SETUP COMPLETE

You now have:
- ✅ All libraries imported
- ✅ Universe defined (50+ tickers)
- ✅ Helper functions ready
- ✅ Test data loaded

**Ready to hunt.**

Next: Run a research notebook (RESEARCH_01, RESEARCH_02, etc.)

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 1: RSI - DOES IT ACTUALLY WORK ON OUR VOLATILE STOCKS?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 1: RSI ANALYSIS")
print("="*80)

# RSI < 30 (Oversold)
oversold = master[master['rsi'] < 30]
print(f"\n📊 RSI < 30 (Oversold): {len(oversold)} events")
print(f"  1-day forward:  avg {oversold['fwd_1d'].mean()*100:+.2f}%, win rate {(oversold['fwd_1d']>0).mean()*100:.1f}%")
print(f"  5-day forward:  avg {oversold['fwd_5d'].mean()*100:+.2f}%, win rate {(oversold['fwd_5d']>0).mean()*100:.1f}%")
print(f"  10-day forward: avg {oversold['fwd_10d'].mean()*100:+.2f}%, win rate {(oversold['fwd_10d']>0).mean()*100:.1f}%")

# RSI > 70 (Overbought)
overbought = master[master['rsi'] > 70]
print(f"\n📊 RSI > 70 (Overbought): {len(overbought)} events")
print(f"  1-day forward:  avg {overbought['fwd_1d'].mean()*100:+.2f}%, win rate {(overbought['fwd_1d']>0).mean()*100:.1f}%")
print(f"  5-day forward:  avg {overbought['fwd_5d'].mean()*100:+.2f}%, win rate {(overbought['fwd_5d']>0).mean()*100:.1f}%")
print(f"  10-day forward: avg {overbought['fwd_10d'].mean()*100:+.2f}%, win rate {(overbought['fwd_10d']>0).mean()*100:.1f}%")

# BASELINE comparison
baseline = master['fwd_5d'].dropna()
print(f"\n📊 BASELINE (Any Random Day): {len(baseline)} events")
print(f"  5-day forward:  avg {baseline.mean()*100:+.2f}%, win rate {(baseline>0).mean()*100:.1f}%")

# VERDICT
print("\n" + "="*80)
rsi_edge = oversold['fwd_5d'].mean() - baseline.mean()
print(f"💡 RSI < 30 EDGE over baseline: {rsi_edge*100:+.2f}%")
if rsi_edge > 0.01:
    print("   ✅ RSI oversold shows POSITIVE edge")
else:
    print("   ❌ RSI oversold does NOT beat random")

ob_edge = overbought['fwd_5d'].mean() - baseline.mean()
print(f"💡 RSI > 70 vs baseline: {ob_edge*100:+.2f}%")
if ob_edge > 0:
    print("   ⚠️ Overbought stocks KEEP GOING UP - don't sell just because RSI high!")
else:
    print("   ✅ Overbought = valid sell signal")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 2: VOLUME SPIKES - WHAT DO THEY MEAN?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 2: VOLUME SPIKE ANALYSIS")
print("="*80)

# Volume > 2x average
vol_spike = master[master['vol_ratio'] > 2].copy()
vol_spike_up = vol_spike[vol_spike['daily_ret'] > 0]
vol_spike_down = vol_spike[vol_spike['daily_ret'] < 0]

print(f"\n📊 Volume > 2x Average: {len(vol_spike)} events")
print(f"  UP days with volume spike: {len(vol_spike_up)}")
print(f"  DOWN days with volume spike: {len(vol_spike_down)}")

print(f"\n📊 After UP day with volume spike:")
print(f"  5-day forward: avg {vol_spike_up['fwd_5d'].mean()*100:+.2f}%, win rate {(vol_spike_up['fwd_5d']>0).mean()*100:.1f}%")

print(f"\n📊 After DOWN day with volume spike:")
print(f"  5-day forward: avg {vol_spike_down['fwd_5d'].mean()*100:+.2f}%, win rate {(vol_spike_down['fwd_5d']>0).mean()*100:.1f}%")

# Test different magnitudes
print("\n📊 Volume Magnitude Testing:")
for mult in [1.5, 2.0, 3.0, 5.0]:
    subset = master[master['vol_ratio'] > mult]
    if len(subset) > 10:
        avg = subset['fwd_5d'].mean()*100
        win = (subset['fwd_5d']>0).mean()*100
        print(f"  Vol > {mult}x: n={len(subset):4d}, 5d return {avg:+.2f}%, win {win:.1f}%")

print("\n" + "="*80)
print("💡 VOLUME INSIGHTS:")
if vol_spike_up['fwd_5d'].mean() > baseline.mean():
    print("   ✅ UP + Volume = momentum continues")
else:
    print("   ❌ UP + Volume = exhaustion, fade it")
    
if vol_spike_down['fwd_5d'].mean() > 0:
    print("   ✅ DOWN + Volume = capitulation, BUY signal")
else:
    print("   ❌ DOWN + Volume = breakdown, avoid")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 3: MOMENTUM - CHASE OR FADE?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 3: MOMENTUM ANALYSIS")
print("="*80)

print("\n📊 After stock UP X% in past 5 days, what happens next 5 days?")
print("-"*70)
for threshold in [5, 10, 15, 20, 30]:
    subset = master[master['ret_5d_past'] > threshold/100]
    if len(subset) > 10:
        avg = subset['fwd_5d'].mean()*100
        win = (subset['fwd_5d']>0).mean()*100
        print(f"  Past 5d > +{threshold}%: n={len(subset):4d}, next 5d {avg:+.2f}%, win {win:.1f}%")

print("\n📊 After stock DOWN X% in past 5 days:")
print("-"*70)
for threshold in [-5, -10, -15, -20, -30]:
    subset = master[master['ret_5d_past'] < threshold/100]
    if len(subset) > 10:
        avg = subset['fwd_5d'].mean()*100
        win = (subset['fwd_5d']>0).mean()*100
        print(f"  Past 5d < {threshold}%: n={len(subset):4d}, next 5d {avg:+.2f}%, win {win:.1f}%")

# Find the sweet spot
print("\n📊 MOMENTUM SWEET SPOT:")
print("-"*70)
ranges = [
    (-30, -20, "Deep dip (-30 to -20%)"),
    (-20, -10, "Moderate dip (-20 to -10%)"),
    (-10, 0, "Slight dip (-10 to 0%)"),
    (0, 10, "Slight up (0 to +10%)"),
    (10, 20, "Moderate run (+10 to +20%)"),
    (20, 30, "Hot (+20 to +30%)"),
    (30, 100, "On fire (+30%+)"),
]
for low, high, label in ranges:
    subset = master[(master['ret_5d_past'] > low/100) & (master['ret_5d_past'] <= high/100)]
    if len(subset) > 20:
        avg = subset['fwd_5d'].mean()*100
        win = (subset['fwd_5d']>0).mean()*100
        print(f"  {label:25s}: n={len(subset):4d}, next 5d {avg:+.2f}%, win {win:.1f}%")

print("\n" + "="*80)
print("💡 MOMENTUM INSIGHTS:")
hot_returns = master[master['ret_5d_past'] > 0.20]['fwd_5d'].mean()
dip_returns = master[(master['ret_5d_past'] < -0.10) & (master['ret_5d_past'] > -0.20)]['fwd_5d'].mean()
if hot_returns > dip_returns:
    print("   ✅ CHASE MOMENTUM - Hot stocks keep running")
else:
    print("   ✅ BUY DIPS - Pullbacks outperform chasing")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 4: SECTOR CORRELATION - DO SYMPATHY PLAYS WORK?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 4: SECTOR CORRELATION ANALYSIS")
print("="*80)

def test_sector_correlation(sector_name):
    """Test if stocks in a sector move together"""
    tickers = UNIVERSE.get(sector_name, [])
    sector_data = master[master['sector'] == sector_name].copy()
    
    if len(sector_data) < 100:
        return None
    
    # Pivot to get returns by ticker
    try:
        pivoted = sector_data.pivot_table(
            values='daily_ret',
            index=sector_data.index.date,
            columns='ticker'
        )
        
        corr = pivoted.corr()
        avg_corr = corr.values[np.triu_indices_from(corr.values, 1)].mean()
        
        return {'sector': sector_name, 'avg_correlation': avg_corr, 'matrix': corr}
    except:
        return None

print("\n📊 SECTOR CORRELATIONS (Do they move together?)")
print("-"*70)

sector_corrs = {}
for sector in UNIVERSE.keys():
    result = test_sector_correlation(sector)
    if result:
        sector_corrs[sector] = result['avg_correlation']
        print(f"  {sector:15s}: avg correlation {result['avg_correlation']:.2f}")

# Find highest correlation sectors (best for sympathy plays)
if sector_corrs:
    sorted_sectors = sorted(sector_corrs.items(), key=lambda x: x[1], reverse=True)
    print("\n💡 SYMPATHY PLAY RANKING (highest correlation = best for sympathy):")
    for sector, corr in sorted_sectors[:5]:
        print(f"   {sector}: {corr:.2f}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 5: REVERSE ENGINEER THE BIG WINNERS
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 5: WHAT DO +20% MOVES LOOK LIKE BEFORE THEY HAPPEN?")
print("="*80)

# Find all +20% moves in 10 days
winners = master[master['fwd_10d'] > 0.20].copy()

print(f"\n📊 Found {len(winners)} instances of +20%+ in 10 days")

print("\n📊 WINNER PROFILE (what they looked like BEFORE the move):")
print("-"*70)

print(f"  Average RSI:              {winners['rsi'].mean():.1f} (vs {master['rsi'].mean():.1f} overall)")
print(f"  Average distance from high: {winners['dist_from_20d_high'].mean()*100:.1f}% (vs {master['dist_from_20d_high'].mean()*100:.1f}%)")
print(f"  Average past 5d momentum:   {winners['ret_5d_past'].mean()*100:+.1f}% (vs {master['ret_5d_past'].mean()*100:+.1f}%)")
print(f"  Average volume ratio:       {winners['vol_ratio'].mean():.2f}x (vs {master['vol_ratio'].mean():.2f}x)")
print(f"  Average volatility:         {winners['volatility_10d'].mean()*100:.2f}% (vs {master['volatility_10d'].mean()*100:.2f}%)")

# What % of winners had these characteristics?
print("\n📊 WINNER CHARACTERISTICS (% that had these traits):")
print("-"*70)
print(f"  RSI < 40:               {(winners['rsi'] < 40).mean()*100:.1f}%")
print(f"  RSI < 50:               {(winners['rsi'] < 50).mean()*100:.1f}%")
print(f"  RSI > 60:               {(winners['rsi'] > 60).mean()*100:.1f}%")
print(f"  Down 10%+ from high:    {(winners['dist_from_20d_high'] < -0.10).mean()*100:.1f}%")
print(f"  Down 20%+ from high:    {(winners['dist_from_20d_high'] < -0.20).mean()*100:.1f}%")
print(f"  Past 5d was positive:   {(winners['ret_5d_past'] > 0).mean()*100:.1f}%")
print(f"  Past 5d was > +10%:     {(winners['ret_5d_past'] > 0.10).mean()*100:.1f}%")
print(f"  Volume > 1.5x average:  {(winners['vol_ratio'] > 1.5).mean()*100:.1f}%")
print(f"  Volume > 2x average:    {(winners['vol_ratio'] > 2).mean()*100:.1f}%")

# Which tickers produced the most winners?
print("\n📊 WHICH TICKERS PRODUCED THE MOST +20% MOVES:")
print("-"*70)
winner_counts = winners.groupby('ticker').size().sort_values(ascending=False)
for ticker, count in winner_counts.head(10).items():
    print(f"  {ticker}: {count} times")

print("\n" + "="*80)
print("💡 WINNER PROFILE INSIGHT:")
if winners['ret_5d_past'].mean() > 0:
    print("   Winners had POSITIVE momentum going in - ride momentum!")
else:
    print("   Winners came from PULLBACKS - buy dips!")
if winners['rsi'].mean() < 50:
    print("   Winners had LOW RSI - oversold was the setup!")
else:
    print("   Winners had HIGH RSI - momentum, not oversold!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 6: DAY OF WEEK - IS MONDAY ACTUALLY BETTER?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 6: DAY OF WEEK ANALYSIS")
print("="*80)

dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

print("\n📊 Returns by Day of Week:")
print("-"*70)
print(f"{'Day':<12} {'Avg Return':<15} {'Win Rate':<12} {'Volatility':<12} {'Count'}")
print("-"*70)

for dow in range(5):
    subset = master[master['dow'] == dow]
    avg = subset['daily_ret'].mean()*100
    win = (subset['daily_ret'] > 0).mean()*100
    vol = subset['daily_ret'].std()*100
    count = len(subset)
    print(f"{dow_names[dow]:<12} {avg:+.3f}%        {win:.1f}%        {vol:.2f}%        {count}")

# Find best day
dow_returns = master.groupby('dow')['daily_ret'].mean()
best_dow = dow_returns.idxmax()
worst_dow = dow_returns.idxmin()

print("\n" + "="*80)
print(f"💡 BEST DAY: {dow_names[best_dow]} ({dow_returns[best_dow]*100:+.3f}%)")
print(f"💡 WORST DAY: {dow_names[worst_dow]} ({dow_returns[worst_dow]*100:+.3f}%)")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 7: GAP ANALYSIS - FADE OR FOLLOW?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 7: GAP ANALYSIS")
print("="*80)

print("\n📊 Gap Up Analysis (Open > Previous Close):")
print("-"*70)

for thresh in [0.03, 0.05, 0.10, 0.15]:
    gap_ups = master[master['gap'] > thresh]
    if len(gap_ups) > 5:
        # What happens by end of day? (intraday)
        intraday = gap_ups['intraday_ret'].mean()*100
        # What happens next day?
        next_day = gap_ups['fwd_1d'].mean()*100
        next_5d = gap_ups['fwd_5d'].mean()*100
        
        print(f"  Gap up > {thresh*100:.0f}%: n={len(gap_ups):4d}")
        print(f"    Same day (intraday):  {intraday:+.2f}% (fade = negative)")
        print(f"    Next day:             {next_day:+.2f}%")
        print(f"    Next 5 days:          {next_5d:+.2f}%")

print("\n📊 Gap Down Analysis:")
print("-"*70)

for thresh in [-0.03, -0.05, -0.10, -0.15]:
    gap_downs = master[master['gap'] < thresh]
    if len(gap_downs) > 5:
        intraday = gap_downs['intraday_ret'].mean()*100
        next_day = gap_downs['fwd_1d'].mean()*100
        next_5d = gap_downs['fwd_5d'].mean()*100
        
        print(f"  Gap down < {thresh*100:.0f}%: n={len(gap_downs):4d}")
        print(f"    Same day (intraday):  {intraday:+.2f}% (bounce = positive)")
        print(f"    Next day:             {next_day:+.2f}%")
        print(f"    Next 5 days:          {next_5d:+.2f}%")

print("\n" + "="*80)
gap_up_5pct = master[master['gap'] > 0.05]['fwd_5d'].mean()
gap_down_5pct = master[master['gap'] < -0.05]['fwd_5d'].mean()
print("💡 GAP INSIGHTS:")
if gap_up_5pct > 0:
    print("   ✅ Gap ups CONTINUE - don't fade, follow!")
else:
    print("   ✅ Gap ups FADE - take profits on gap ups")
if gap_down_5pct > 0:
    print("   ✅ Gap downs BOUNCE - buy the panic!")
else:
    print("   ❌ Gap downs CONTINUE - avoid catching falling knives")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 8: PULLBACK SWEET SPOT - HOW FAR FROM HIGH IS BEST?
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 8: DISTANCE FROM HIGH ANALYSIS")
print("="*80)

print("\n📊 Where should we buy? (Distance from 20-day high)")
print("-"*70)

ranges = [
    (0, -0.05, "At highs (0 to -5%)"),
    (-0.05, -0.10, "Slight dip (-5% to -10%)"),
    (-0.10, -0.20, "Pullback (-10% to -20%)"),
    (-0.20, -0.30, "Correction (-20% to -30%)"),
    (-0.30, -0.40, "Deep pullback (-30% to -40%)"),
    (-0.40, -0.50, "Crash (-40% to -50%)"),
    (-0.50, -1.0, "Destroyed (-50%+)"),
]

best_zone = None
best_return = -999

for high_bound, low_bound, label in ranges:
    subset = master[(master['dist_from_20d_high'] <= high_bound) & 
                    (master['dist_from_20d_high'] > low_bound)]
    if len(subset) > 20:
        avg = subset['fwd_5d'].mean()*100
        win = (subset['fwd_5d']>0).mean()*100
        print(f"  {label:25s}: n={len(subset):4d}, 5d return {avg:+.2f}%, win {win:.1f}%")
        
        if avg > best_return:
            best_return = avg
            best_zone = label

print("\n" + "="*80)
print(f"💡 BEST BUY ZONE: {best_zone}")
print(f"   Expected 5-day return: {best_return:+.2f}%")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 9: COMBINED SIGNALS - STACKING EDGES
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 9: COMBINED SIGNALS")
print("="*80)

def score_day(row):
    """Score each day based on multiple signals"""
    score = 0
    
    # RSI oversold
    if row['rsi'] < 40:
        score += 1
    if row['rsi'] < 30:
        score += 1
    
    # Pullback from high
    if row['dist_from_20d_high'] < -0.10:
        score += 1
    if row['dist_from_20d_high'] < -0.20:
        score += 1
        
    # Volume confirmation
    if row['vol_ratio'] > 1.5:
        score += 1
    if row['vol_ratio'] > 2.0:
        score += 1
        
    # Positive momentum starting (but not extended)
    if row['ret_5d_past'] > 0 and row['ret_5d_past'] < 0.15:
        score += 1
    
    # Above moving average (trend)
    if row['above_sma20']:
        score += 1
    
    return score

master['signal_score'] = master.apply(score_day, axis=1)

print("\n📊 Signal Score vs Forward Returns:")
print("-"*70)
print(f"{'Score':<8} {'Count':<10} {'5d Return':<15} {'Win Rate'}")
print("-"*70)

for score in range(0, 9):
    subset = master[master['signal_score'] == score]
    if len(subset) > 20:
        avg = subset['fwd_5d'].mean()*100
        win = (subset['fwd_5d']>0).mean()*100
        print(f"{score:<8} {len(subset):<10} {avg:+.2f}%          {win:.1f}%")

# Test high score vs low score
high_score = master[master['signal_score'] >= 4]
low_score = master[master['signal_score'] <= 1]

print("\n" + "="*80)
print("💡 STACKING SIGNALS:")
print(f"   High score (4+): {high_score['fwd_5d'].mean()*100:+.2f}%, win rate {(high_score['fwd_5d']>0).mean()*100:.1f}% (n={len(high_score)})")
print(f"   Low score (0-1): {low_score['fwd_5d'].mean()*100:+.2f}%, win rate {(low_score['fwd_5d']>0).mean()*100:.1f}% (n={len(low_score)})")
edge = high_score['fwd_5d'].mean() - low_score['fwd_5d'].mean()
print(f"   EDGE FROM STACKING: {edge*100:+.2f}%")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 10: PATTERNS HUMANS MIGHT MISS
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 10: HIDDEN PATTERNS - What humans miss")
print("="*80)

# PATTERN 1: Volatility Contraction -> Expansion
print("\n📊 PATTERN 1: Volatility Contraction (calm before storm)")
print("-"*70)
master['vol_contracting'] = master['volatility_10d'] < master['volatility_20d'] * 0.7
contracted = master[master['vol_contracting']]
print(f"  When volatility contracts (10d vol < 70% of 20d vol):")
print(f"    Count: {len(contracted)}")
print(f"    Next 5d return: {contracted['fwd_5d'].mean()*100:+.2f}%")
print(f"    Win rate: {(contracted['fwd_5d']>0).mean()*100:.1f}%")

# PATTERN 2: Consecutive Days
print("\n📊 PATTERN 2: After Consecutive Up/Down Days")
print("-"*70)

def count_streak(df):
    streaks = []
    current_streak = 0
    for ret in df['daily_ret']:
        if pd.isna(ret):
            streaks.append(0)
        elif ret > 0:
            current_streak = max(1, current_streak + 1) if current_streak > 0 else 1
            streaks.append(current_streak)
        else:
            current_streak = min(-1, current_streak - 1) if current_streak < 0 else -1
            streaks.append(current_streak)
    return streaks

# Apply to each ticker
master['streak'] = master.groupby('ticker').apply(
    lambda x: pd.Series(count_streak(x), index=x.index)
).values.flatten() if 'ticker' in master.columns else 0

three_up = master[master['streak'] >= 3]
three_down = master[master['streak'] <= -3]

print(f"  After 3+ green days: n={len(three_up)}, next day {three_up['fwd_1d'].mean()*100:+.2f}%")
print(f"  After 3+ red days: n={len(three_down)}, next day {three_down['fwd_1d'].mean()*100:+.2f}%")

# PATTERN 3: Volume Exhaustion (Big move on declining volume)
print("\n📊 PATTERN 3: Volume Exhaustion (move without volume confirmation)")
print("-"*70)
exhaustion = master[(master['ret_5d_past'] > 0.15) & (master['vol_trend'] < 0.8)]
confirmed = master[(master['ret_5d_past'] > 0.15) & (master['vol_trend'] > 1.2)]
print(f"  +15% on DECLINING volume (exhaustion): n={len(exhaustion)}, next 5d {exhaustion['fwd_5d'].mean()*100:+.2f}%")
print(f"  +15% on INCREASING volume (confirmed): n={len(confirmed)}, next 5d {confirmed['fwd_5d'].mean()*100:+.2f}%")

# PATTERN 4: Distance from SMA
print("\n📊 PATTERN 4: Extreme Extension from 20 SMA")
print("-"*70)
very_extended = master[master['dist_from_sma20'] > 0.20]
very_oversold = master[master['dist_from_sma20'] < -0.20]
print(f"  20%+ ABOVE SMA20 (extended): n={len(very_extended)}, next 5d {very_extended['fwd_5d'].mean()*100:+.2f}%")
print(f"  20%+ BELOW SMA20 (oversold): n={len(very_oversold)}, next 5d {very_oversold['fwd_5d'].mean()*100:+.2f}%")

# PATTERN 5: Monday following a big week
print("\n📊 PATTERN 5: What happens on Monday after big weeks?")
print("-"*70)
# This is a proxy - Monday after 5d was up big
big_week_then_monday = master[(master['ret_5d_past'].shift(1) > 0.10) & master['is_monday']]
print(f"  Monday after +10% week: n={len(big_week_then_monday)}")
if len(big_week_then_monday) > 5:
    print(f"    Monday return: {big_week_then_monday['daily_ret'].mean()*100:+.2f}%")

print("\n" + "="*80)
print("💡 HIDDEN PATTERN INSIGHTS - What humans miss:")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🐺 TEST 11: REVERSE ENGINEER SPECIFIC WINNERS (MU, APLD, OKLO)
# ═══════════════════════════════════════════════════════════════════════════════

print("="*80)
print("🔬 TEST 11: REVERSE ENGINEERING MU, APLD, OKLO")
print("="*80)

def reverse_engineer_winner(ticker):
    """Look at what the stock looked like before its big moves"""
    df = master[master['ticker'] == ticker].copy()
    
    if len(df) == 0:
        print(f"  {ticker}: No data")
        return
    
    # Find all +10% days
    big_days = df[df['daily_ret'] > 0.10]
    
    print(f"\n📊 {ticker} - Found {len(big_days)} days with +10%+ moves:")
    
    for idx, row in big_days.iterrows():
        # Get the day before
        loc = df.index.get_loc(idx)
        if loc > 0:
            prev = df.iloc[loc-1]
            print(f"\n  📅 {idx.date()}: +{row['daily_ret']*100:.1f}%")
            print(f"     RSI day before:    {prev['rsi']:.1f}")
            print(f"     Volume ratio:      {row['vol_ratio']:.2f}x")
            print(f"     From 20d high:     {prev['dist_from_20d_high']*100:.1f}%")
            print(f"     Past 5d momentum:  {prev['ret_5d_past']*100:+.1f}%")

for ticker in ['MU', 'APLD', 'OKLO', 'AEVA', 'VST']:
    if ticker in master['ticker'].values:
        reverse_engineer_winner(ticker)
    else:
        print(f"\n⚠️  {ticker}: Not in dataset")

print("\n" + "="*80)
print("💡 Use these profiles to identify SIMILAR setups in other stocks!")

---

# 🧠 TRADING WISDOM TO TEST

## The "Known Truths" That Need Proof

Everything below is stuff that traders say, books teach, or seems logical.
**But does it actually work on OUR stocks in the LAST 3 months?**

We're not taking anyone's word for it. We test EVERYTHING.

---

## 📚 CATEGORY 1: Technical Indicator "Rules"

### RSI (Relative Strength Index)
**What they say:**
- RSI < 30 = oversold = buy opportunity
- RSI > 70 = overbought = sell/short opportunity
- RSI divergence (price makes new low, RSI doesn't) = reversal coming

**The maybes:**
- Maybe RSI < 30 works on big caps but fails on volatile small caps?
- Maybe RSI < 20 is the real signal (more extreme)?
- Maybe RSI > 70 doesn't mean sell on momentum stocks - they stay overbought for weeks?
- Maybe RSI works better when COMBINED with volume confirmation?
- Maybe RSI crossing back ABOVE 30 (not just being below) is the real signal?

**Test ideas:**
```python
# Standard RSI < 30
# RSI < 25 vs < 30 vs < 35
# RSI crossing UP through 30 vs just being below 30
# RSI < 30 + volume spike
# RSI < 30 + sector green
# RSI divergence detection
```

---

### Moving Averages
**What they say:**
- Price above 50 MA = bullish
- Price below 50 MA = bearish
- Golden cross (50 crosses above 200) = major bull signal
- Death cross (50 crosses below 200) = major bear signal
- 20 MA is "fast" trend, 50 is "medium", 200 is "slow"

**The maybes:**
- Maybe MAs are too slow for volatile stocks that move 10% in a day?
- Maybe the 10 or 8 period MA is more relevant for our fast movers?
- Maybe distance FROM the MA matters more (extended above = pullback coming)?
- Maybe MA works as support on pullbacks but not as signal to buy?

**Test ideas:**
```python
# Price 10%+ above 20 MA = mean revert down?
# Price touches 20 MA from above = bounce?
# Golden/death cross on our stocks - lagging indicator?
```

---

### Volume
**What they say:**
- Volume confirms price movement
- Breakout on high volume = real, breakout on low volume = fake
- Climax volume (huge spike) = exhaustion, reversal coming
- Accumulation = up days on higher volume than down days
- Distribution = down days on higher volume than up days

**The maybes:**
- Maybe 2x volume isn't enough - need 3x or 5x for signal?
- Maybe volume spike UP day vs DOWN day mean completely different things?
- Maybe declining volume on an uptrend is actually fine (path of least resistance)?
- Maybe the first high volume day after quiet period is the signal (accumulation complete)?

**Test ideas:**
```python
# Volume 2x vs 3x vs 5x average - which matters?
# High volume UP day - continuation or exhaustion?
# High volume DOWN day - capitulation (buy) or breakdown (sell)?
# Volume trend (5 days increasing vs decreasing)
# Volume on breakout to new highs
```

---

### Support/Resistance
**What they say:**
- Previous highs become resistance
- Previous lows become support
- Round numbers ($10, $50, $100) act as psychological levels
- Once support breaks, it becomes resistance (and vice versa)

**The maybes:**
- Maybe S/R doesn't work on volatile stocks that gap through levels?
- Maybe only RECENT S/R matters (last 20 days vs 52 weeks)?
- Maybe S/R works for entries but not exits on momentum stocks?
- Maybe we should BUY breakouts above resistance instead of fading them?

**Test ideas:**
```python
# Price near 52-week high - continuation or resistance?
# Price near 52-week low - support or breakdown?
# Round number analysis ($5, $10, $20 levels)
# Previous day high/low as intraday S/R
```

---

## 📚 CATEGORY 2: Chart Patterns

### Gap Behavior
**What they say:**
- "Gaps get filled" - gaps eventually return to pre-gap price
- Gap up = profit taking, fade it
- Gap down = panic, buy the dip
- "Gap and go" = if gap holds first 30 min, trend continues

**The maybes:**
- Maybe gap ups in momentum stocks DON'T fill - they keep running?
- Maybe small gaps (2-3%) fade but large gaps (10%+) continue?
- Maybe gap direction matters less than VOLUME on the gap?
- Maybe gaps on news are different than gaps on no news?

**Test ideas:**
```python
# Gap up 5%+ - does it fade same day?
# Gap down 5%+ - does it bounce same day?
# Gap size vs fill probability
# Gap + volume analysis
# Gap on Monday vs other days
```

---

### Candlestick Patterns
**What they say:**
- Doji = indecision, reversal coming
- Hammer at bottom = bullish reversal
- Shooting star at top = bearish reversal
- Engulfing patterns = strong reversal signal
- Three white soldiers / three black crows = trend continuation

**The maybes:**
- Maybe candlestick patterns are noise on daily charts for volatile stocks?
- Maybe only LARGE candles matter (not small indecision)?
- Maybe patterns work on higher timeframes (weekly) not daily?
- Maybe patterns need volume confirmation to be valid?

**Test ideas:**
```python
# Large red candle (>5% down) - bounce or continuation?
# Large green candle (>5% up) - continuation or exhaustion?
# Candle size vs next day direction
# Body vs wick ratio meaning
```

---

## 📚 CATEGORY 3: Momentum & Mean Reversion

### Momentum
**What they say:**
- "Trend is your friend" - ride winners
- Momentum persists - winners keep winning short term
- "Let winners run, cut losers fast"
- Relative strength (vs market) predicts future performance

**The maybes:**
- Maybe momentum works for 3-5 days then exhausts?
- Maybe there's a "sweet spot" - not too hot, not too cold?
- Maybe momentum in our volatile stocks is shorter than blue chips?
- Maybe momentum ONLY works with volume confirmation?

**Test ideas:**
```python
# Stock up 10% last 5 days - next 5 days?
# Stock up 20% last 5 days - continuation or reversal?
# Momentum + RSI not overbought = better?
# Sector momentum vs individual momentum
```

---

### Mean Reversion
**What they say:**
- "What goes up must come down"
- Extended moves revert to the mean
- 2 standard deviations from average = extreme, will revert
- Rubber band effect - stretched too far, snaps back

**The maybes:**
- Maybe mean reversion works on range-bound stocks but not trending?
- Maybe our stocks trend more than they mean-revert?
- Maybe you need BOTH extension AND exhaustion signal?
- Maybe mean reversion is a losing strategy in bull markets?

**Test ideas:**
```python
# Stock 2+ std devs above 20 MA - revert?
# Stock 30%+ above 20-day average price - pullback coming?
# Distance from recent high vs future returns
```

---

## 📚 CATEGORY 4: Time-Based Patterns

### Day of Week
**What they say:**
- "Sell on Monday" - weekend worry leads to selling
- "Tuesday reversal" - Monday sellers exhausted
- "Friday profit taking" - don't hold into weekend
- "Turnaround Tuesday" - most common reversal day

**The maybes:**
- Maybe day of week effects existed but are arbitraged away?
- Maybe they only work on certain stock types?
- Maybe our stocks have OPPOSITE patterns (Monday morning buy)?
- Maybe it's hour of day that matters more than day of week?

**Test ideas:**
```python
# Average return by day of week
# Volatility by day of week
# Best day to BUY vs best day to SELL
# Monday open vs Friday close
```

---

### Time of Day (Intraday)
**What they say:**
- First 30 min = "amateur hour" - volatile, reversals common
- 10am reversal - early trend often reverses around 10am
- Lunch hour = low volume, choppy, avoid trading
- Power hour (3-4pm) = real moves, institutional activity
- Last 10 min = positioning for next day

**The maybes:**
- Maybe the "amateur hour" is where we SHOULD buy (volatility = opportunity)?
- Maybe power hour is too late - smart money moved already?
- Maybe our small caps have different intraday patterns than SPY?

**Test ideas:**
```python
# First 30 min direction vs rest of day
# Buying at open vs buying at 10am
# Power hour volume vs morning volume
```

---

### Seasonal/Calendar
**What they say:**
- "Sell in May, go away" - summer months underperform
- January effect - small caps outperform in January
- Santa rally - end of year bullish
- Options expiration week = volatility
- Earnings season = increased volatility

**The maybes:**
- Maybe these effects are too slow for our trading style?
- Maybe knowing earnings dates matters more than season?
- Maybe we should trade MORE during high volatility periods, not less?

---

## 📚 CATEGORY 5: Sentiment & Psychology

### Fear/Greed
**What they say:**
- "Be fearful when others are greedy, greedy when others are fearful"
- VIX spike = buy opportunity (fear extreme)
- Everyone bullish = contrarian sell signal
- Capitulation (panic selling) = bottom near

**The maybes:**
- Maybe VIX doesn't affect small caps directly?
- Maybe our stocks have their own "mini VIX" (sector fear)?
- Maybe capitulation volume is hard to identify in real-time?

**Test ideas:**
```python
# VIX spike days - our stocks next 5 days?
# Market down 2%+ - our stocks bounce more/less than SPY?
```

---

### News/Catalysts
**What they say:**
- "Buy the rumor, sell the news"
- Good earnings = stock goes up
- Bad earnings = stock goes down
- Analyst upgrades/downgrades move stocks
- Insider buying = bullish signal

**The maybes:**
- Maybe "sell the news" is wrong for small caps with real catalysts?
- Maybe good earnings are already priced in (expectation matters)?
- Maybe insider buying is lagging (they buy months before)?
- Maybe analyst coverage is too sparse for small caps?

**Test ideas:**
```python
# Price action 5 days before vs 5 days after 8-K filings
# Earnings date analysis (run-up, reaction, post-earnings drift)
# Form 4 cluster detection (multiple insiders buying)
```

---

## 📚 CATEGORY 6: Sector & Correlation

### Sector Behavior
**What they say:**
- "A rising tide lifts all boats" - sector moves together
- Leaders lead, laggards lag then catch up
- Sector rotation - money flows from one sector to another
- When SPY tanks, correlations go to 1 (everything sells)

**The maybes:**
- Maybe our sectors (AI, defense, quantum) are too niche for normal rules?
- Maybe one stock CAN move while sector is flat (individual catalysts)?
- Maybe laggards STAY laggards (don't catch up)?
- Maybe sector ETF direction predicts individual stock direction?

**Test ideas:**
```python
# AEVA +20% - what happens to OUST, INVZ next day?
# Sector average return vs individual stock forward return
# Leader identification - who moves first?
# Beta to SPY by ticker
```

---

### Correlation Breakdown
**What they say:**
- Diversification reduces risk
- Uncorrelated assets protect portfolio
- During crisis, correlations spike (no hiding place)

**The maybes:**
- Maybe our stocks are SO correlated that diversification is illusion?
- Maybe we should embrace correlation - go all-in on best sector?
- Maybe negative correlation exists within our universe (AI vs defense)?

---

## 📚 CATEGORY 7: Position Sizing & Risk

### Position Sizing Rules
**What they say:**
- Never risk more than 1-2% of portfolio per trade
- Scale into positions (don't buy all at once)
- Add to winners, not losers (pyramiding)
- Equal weight vs conviction weighting

**The maybes:**
- Maybe 1-2% is too conservative for small accounts?
- Maybe conviction weighting beats equal weight?
- Maybe scaling in misses the move on volatile stocks?
- Maybe adding to losers (averaging down) works if thesis intact?

---

### Stop Losses
**What they say:**
- Always use stop losses
- 5-10% stop for volatile stocks
- Trailing stops to lock in profits
- Stop at support level, not arbitrary %

**The maybes:**
- Maybe stops get hunted on volatile stocks (stop runs)?
- Maybe mental stops better than hard stops?
- Maybe wider stops (15-20%) make sense for our stocks?
- Maybe time-based stops (sell after X days regardless)?

---

## 📚 CATEGORY 8: Market Regime

### Bull vs Bear Market
**What they say:**
- "Don't fight the fed" - monetary policy drives markets
- Bull markets climb a wall of worry
- Bear markets have sharp rallies that fail
- Different strategies for different regimes

**The maybes:**
- Maybe our growth stocks do better when rates are expected to fall?
- Maybe small caps lead into bull markets and lag into bear?
- Maybe we need regime detection before applying signals?

---

### Volatility Regime
**What they say:**
- Low vol begets high vol (calm before storm)
- High vol = opportunity for active traders
- Volatility clusters - volatile days cluster together
- Mean reversion works better in high vol regimes

**The maybes:**
- Maybe our stocks are ALWAYS high vol, so regime doesn't matter?
- Maybe we should size positions based on recent volatility?
- Maybe some signals only work in certain vol environments?

**Test ideas:**
```python
# Define high/low vol regime (rolling 20-day std)
# Test all signals within each regime
# Signals that work in high vol vs low vol
```

---

## 🐺 THE "CRAZY IDEAS" TO TEST

These might be wrong. But what if they're right?

1. **Inverse RSI** - What if RSI > 70 is actually BUY on momentum stocks?
2. **Volume Exhaustion** - What if massive volume (5x) means move is DONE?
3. **Gap Continuation** - What if large gaps (10%+) should be CHASED not faded?
4. **Sympathy Lag** - What if buying the 2nd mover in sector beats buying the leader?
5. **Breakdown Buy** - What if buying NEW 52-week lows is better than buying pullbacks?
6. **News Fade Inverse** - What if good news on small caps should be BOUGHT not sold?
7. **Options Flow** - What if unusual options activity predicts next day direction?
8. **Short Interest** - What if high short interest (>20%) means squeeze potential?
9. **Insider Clusters** - What if 3+ insiders buying same week is the real signal?
10. **SEC Filing Lag** - What if market takes 24-48 hours to digest 8-K filings?

---

## 📊 THE RESEARCH MATRIX

For EACH hypothesis, we need to answer:

| Question | How We Measure |
|----------|---------------|
| Does it work? | Win rate > 50%, avg return > 0 |
| Is it BETTER than random? | Compare to baseline (any random day) |
| How MUCH better? | Excess return over baseline |
| Is sample size enough? | n > 30 events ideally |
| Does it work on ALL tickers? | Test by ticker |
| Does it work RECENTLY? | Last 1 month vs last 3 months |
| What makes it work BETTER? | Combinations with other signals |
| What makes it FAIL? | Identify false signal conditions |

---

## ⏰ WHY 3 MONTHS IS RIGHT

For our stocks (volatile small caps), 3 months is:

✅ **Recent enough** - Market regime hasn't changed dramatically
✅ **Long enough** - Get ~65 trading days of data
✅ **Relevant data** - These stocks change fast, old data is noise
✅ **Multiple cycles** - Capture rallies AND pullbacks

**NOT 1 month:** Too few data points, might be in unusual period
**NOT 6 months:** Might include regime change, old patterns
**NOT 1 year:** These stocks evolve fast, year-old data is irrelevant

For our universe of 50 tickers × 65 days = ~3,250 data points
That's enough to find real patterns IF they exist.

---

## 🎯 THE GOAL

After testing all of this, we should have:

1. **VALIDATED EDGES** - Signals that actually work (>60% win rate or positive expectancy)
2. **KILLED ASSUMPTIONS** - "Rules" that don't work on our stocks
3. **CONDITIONAL EDGES** - Signals that work only in certain conditions
4. **COMBINED SIGNALS** - Stacking that improves results
5. **TICKER-SPECIFIC** - Which stocks respond to which signals

Then we build tools around PROVEN edges only.

No more "RSI said buy so I bought" without knowing if RSI actually works on that stock.

---

🐺 **This is the knowledge base. Now we test it ALL.**